# Intermediate Scraping Homework: Wikipedia Table

In this assignment, we'll be extracting data from Wikipedia's table of the tallest buildings in Brooklyn: https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_Brooklyn

### 0) Setup

Import `requests`, `BeautifulSoup`, and `pandas`. Although this homework uses `BeautifulSoup`, you can choose to use `lxml` instead, if you prefer.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### 1) Grab the HTML for the webpage linked above

Use `requests` to get the HTML, assigning it to a variable

In [2]:
import requests
http_response=requests.get('https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_Brooklyn')
html=http_response.text

### 2) Convert the HTML into a `BeautifulSoup` object

In [3]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(html)

### 3) Use `.select(...)` (and `[0]`) to select the main table

That's the one directly under the "Tallest buildings" heading.

Print out the first 100 characters of text from the table to make sure you have the right one.

In [139]:
main_table=soup.select("table.wikitable")[0]
main_table.text[:100]

'\n\nRank\n\nName[a]\n\nImage\n\nHeightft (m)\n\nFloors\n\nYear completed\n\nNotes\n\n\n1\n\nThe Brooklyn Tower\n\n\n\n1,066'

### 4) Use `.select(...)` (and `[0]`) again to select the table's first row

... which is its header. (Reminder that the `<thead>` tag is optional. Wikipedia tables don't use it.)

In [45]:
header=main_table.select("th")

### 5) Extract the column names from that header

Use `.strip()` to remove any leading or trailing whitespace from the names.

First, try doing this with a standard `for` loop:

In [49]:
for head in header:
    print (head.text.strip())

Rank
Name[a]
Image
Heightft (m)
Floors
Year completed
Notes


Try to do the same, but with a list comprehension:

In [48]:
header = main_table.select("th")
col_names = [header.text.strip() for header in header]
col_names

['Rank',
 'Name[a]',
 'Image',
 'Heightft (m)',
 'Floors',
 'Year completed',
 'Notes']

### 6) Select all non-header row *elements* from the table

Since the header was the first row, you'll want to skip that one. How many rows are there? (Check with your eyes that this number matches what you deduce from the rankings column in the browser-rendered table.)

In [180]:
row_els= main_table.select("tbody tr")[1:]

### 7) For each row, extract the text of each cell into a Python list

First, try this as two nested `for` loops:

In [165]:
tall_bldg = []
for row in row_els:
    row_cells = []
    for cell in row.select("td"):
        row_cells.append(cell.text)
    tall_bldg.append(row_cells)

tall_bldg

[['1\n',
  'The Brooklyn Tower\n',
  '\n',
  '1,066 (325)\n',
  '93\n',
  '2022\n',
  'Topped out in October 2021.[2][23][24][25]\n'],
 ['2\n',
  'Brooklyn Point\n',
  '\n',
  '720 (219)\n',
  '68\n',
  '2019\n',
  "The final phase of Extell's City Point development; topped out in April 2019, it is now the second tallest building in Brooklyn.[26] Also known as 138 Willoughby Street,[27][28] 1 City Point,[29] and City Point Tower III.[29][30][31]\n"],
 ['3\n',
  '11 Hoyt\n',
  '\n',
  '626 (191)\n',
  '51\n',
  '2020\n',
  "Topped out in June 2019.[32] A redevelopment of Macy's former footprint in Downtown Brooklyn, with a design seemingly inspired by 8 Spruce Street.[33][34]\n"],
 ['4\n',
  'The Hub\n',
  '\n',
  '611 (186)\n',
  '52\n',
  '2017\n',
  'Also known as 333 Schermerhorn Street. Topped out on December 16, 2015.[35][36][37][38][39]\n'],
 ['5\n',
  '98 Dekalb Avenue\n',
  'Upload image\n',
  '610 (186)\n',
  '49\n',
  '2025\n',
  'Topped out in June 2024.[40][41]\n'],
 ['6\n'

Try the same, but with two list comprehensions (one nested in the other):

In [166]:
tall_bldg = [
    [cell.text.strip() for cell in row.select("td")]
    for row in row_els
]
tall_bldg

[['1',
  'The Brooklyn Tower',
  '',
  '1,066 (325)',
  '93',
  '2022',
  'Topped out in October 2021.[2][23][24][25]'],
 ['2',
  'Brooklyn Point',
  '',
  '720 (219)',
  '68',
  '2019',
  "The final phase of Extell's City Point development; topped out in April 2019, it is now the second tallest building in Brooklyn.[26] Also known as 138 Willoughby Street,[27][28] 1 City Point,[29] and City Point Tower III.[29][30][31]"],
 ['3',
  '11 Hoyt',
  '',
  '626 (191)',
  '51',
  '2020',
  "Topped out in June 2019.[32] A redevelopment of Macy's former footprint in Downtown Brooklyn, with a design seemingly inspired by 8 Spruce Street.[33][34]"],
 ['4',
  'The Hub',
  '',
  '611 (186)',
  '52',
  '2017',
  'Also known as 333 Schermerhorn Street. Topped out on December 16, 2015.[35][36][37][38][39]'],
 ['5',
  '98 Dekalb Avenue',
  'Upload image',
  '610 (186)',
  '49',
  '2025',
  'Topped out in June 2024.[40][41]'],
 ['6',
  'AVA DoBro',
  '',
  '596 (182)',
  '58',
  '2015',
  'Also known as

### 8) Turn the data you've extracted into a `pandas` `DataFrame`

In [167]:
import pandas as pd
bldg_df = pd.DataFrame(tall_bldg, columns=col_names)
bldg_df

,Rank,Name[a],Image,Heightft (m),Floors,Year completed,Notes
0,1,The Brooklyn Tower,,"1,066 (325)",93,2022,Topped out in October 2021.[2][23][24][25]
1,2,Brooklyn Point,,720 (219),68,2019,The final phase of Extell's City Point develop...
2,3,11 Hoyt,,626 (191),51,2020,Topped out in June 2019.[32] A redevelopment o...
3,4,The Hub,,611 (186),52,2017,Also known as 333 Schermerhorn Street. Topped ...
4,5,98 Dekalb Avenue,Upload image,610 (186),49,2025,Topped out in June 2024.[40][41]
...,...,...,...,...,...,...,...
75,76,595 Dean Street,Upload image,298 (91),27,2022,Topped out in February 2022.[178][179]
76,77 =,Beacon Tower,,297 (91),23,2007,[180][181]
77,77 =,One Northside Piers,,297 (91),29,2008,[182][183]
78,77 =,101 Clark Street,Upload image,295 (90),30,1973,[184][185]


### 9) Which years are represented by at least 5 buildings?

In [179]:
bldg_df['Year completed'].value_counts().head(4)

Year completed
2022    11
2020     6
2016     6
2024     5
Name: count, dtype: int64

### 10) How many total floors do all the buildings have, combined?

In [174]:
bldg_df['Floors'].astype(int).sum()

2905

### 11) How many of the buildings have their own Wikipedia page?

For this, you'll need to query the row elements again; the information won't have been extracted into your `DataFrame`. 

(Hint: Whether a building has its own Wikipedia page isn't an explicit piece of data, but something you can infer from the presence of a particular sub-element.)

In [338]:
links=[]
for row in row_els:
    title=row.select('td')[1]
    if title.select('a'):
            links.append(title)

In [337]:
len(links)

18

### 12) How many have an image?

You could do this by testing for the presence of another element:

In [309]:
images=[]
for row in row_els:
    if (row.find('img')):
        images.append(row.find('img'))
len(images)

61

Or through information that's already in your `DataFrame`:

In [311]:
bldg_df['Image'].value_counts()

Image
                61
Upload image    19
Name: count, dtype: int64

### Bonus challenge

If we tried to run the same code on https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_New_York_City instead, the results wouldn't be quite right. Try it. Then, examining the HTML of that page, try to figure out why.

If you want an extra-extra challenge, try writing code that would parse that table correctly.

In [ ]:
#not right now lol

---

---

---